## Part 3: Predictive Modeling
Ultimate is interested in predicting rider retention. To help explore this question, a sample dataset of a cohort of users who signed up for an Ultimate account in January 2014 is provided in 'ultimate_data_challenge.json'. The data was pulled several months later, and a user is considered retained if they were “active” in the preceding 30 days. 
<br>My task is to use this dataset to help understand what factors are the best predictors for retention, and offer suggestions to operationalize those insights to help Ultimate.

In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

In [71]:
path = "../Takehome_challenge_3/ultimate_data_challenge.json"

In [72]:
df = pd.read_json(path)
df.head()

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver
0,King's Landing,4,2014-01-25,4.7,1.10,2014-06-17,iPhone,15.4,True,46.2,3.67,5.0
1,Astapor,0,2014-01-29,5.0,1.00,2014-05-05,Android,0.0,False,50.0,8.26,5.0
2,Astapor,3,2014-01-06,4.3,1.00,2014-01-07,iPhone,0.0,False,100.0,0.77,5.0
3,King's Landing,9,2014-01-10,4.6,1.14,2014-06-29,iPhone,20.0,True,80.0,2.36,4.9
4,Winterfell,14,2014-01-27,4.4,1.19,2014-03-15,Android,11.8,False,82.4,3.13,4.9


In [73]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    50000 non-null  object 
 1   trips_in_first_30_days  50000 non-null  int64  
 2   signup_date             50000 non-null  object 
 3   avg_rating_of_driver    41878 non-null  float64
 4   avg_surge               50000 non-null  float64
 5   last_trip_date          50000 non-null  object 
 6   phone                   49604 non-null  object 
 7   surge_pct               50000 non-null  float64
 8   ultimate_black_user     50000 non-null  bool   
 9   weekday_pct             50000 non-null  float64
 10  avg_dist                50000 non-null  float64
 11  avg_rating_by_driver    49799 non-null  float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


In [74]:
df.describe()

,trips_in_first_30_days,avg_rating_of_driver,avg_surge,surge_pct,weekday_pct,avg_dist,avg_rating_by_driver
count,50000.000000,41878.000000,50000.000000,50000.000000,50000.000000,50000.000000,49799.000000
mean,2.278200,4.601559,1.074764,8.849536,60.926084,5.796827,4.778158
std,3.792684,0.617338,0.222336,19.958811,37.081503,5.707357,0.446652
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,4.300000,1.000000,0.000000,33.300000,2.420000,4.700000
50%,1.000000,4.900000,1.000000,0.000000,66.700000,3.880000,5.000000
75%,3.000000,5.000000,1.050000,8.600000,100.000000,6.940000,5.000000
max,125.000000,5.000000,8.000000,100.000000,100.000000,160.960000,5.000000


There is some cleaning to do: I need to cast signup_date and last_trip_date to datetime type and decide how to handle the nulls in avg_rating_of_driver, phone, and avg_rating_by_driver. And first I will create a binary int column for whether a user is retained or not.

In [75]:
df["retained"]=np.where(df['trips_in_first_30_days']>=1, 1, 0)

In [76]:
df.head()

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver,retained
0,King's Landing,4,2014-01-25,4.7,1.10,2014-06-17,iPhone,15.4,True,46.2,3.67,5.0,1
1,Astapor,0,2014-01-29,5.0,1.00,2014-05-05,Android,0.0,False,50.0,8.26,5.0,0
2,Astapor,3,2014-01-06,4.3,1.00,2014-01-07,iPhone,0.0,False,100.0,0.77,5.0,1
3,King's Landing,9,2014-01-10,4.6,1.14,2014-06-29,iPhone,20.0,True,80.0,2.36,4.9,1
4,Winterfell,14,2014-01-27,4.4,1.19,2014-03-15,Android,11.8,False,82.4,3.13,4.9,1


In [77]:
df.value_counts(subset='retained')

retained
1    34610
0    15390
dtype: int64

In [78]:
#calculate % retained users
retained = df['retained'].value_counts()[1]
not_retained  = df['retained'].value_counts()[0]

pct = retained/(retained+not_retained)*100
print(f"Percent users retained: {pct}%")

Percent users retained: 69.22%


In [79]:
#cast date columns from objects to datetime
df['signup_date'] = pd.to_datetime(df['signup_date'])
df['last_trip_date'] = pd.to_datetime(df['last_trip_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   city                    50000 non-null  object        
 1   trips_in_first_30_days  50000 non-null  int64         
 2   signup_date             50000 non-null  datetime64[ns]
 3   avg_rating_of_driver    41878 non-null  float64       
 4   avg_surge               50000 non-null  float64       
 5   last_trip_date          50000 non-null  datetime64[ns]
 6   phone                   49604 non-null  object        
 7   surge_pct               50000 non-null  float64       
 8   ultimate_black_user     50000 non-null  bool          
 9   weekday_pct             50000 non-null  float64       
 10  avg_dist                50000 non-null  float64       
 11  avg_rating_by_driver    49799 non-null  float64       
 12  retained                50000 non-null  int32 

Now to handle the missing data. There are few enough missing values in the phone column that it should be fine to just impute with the mode. And likewise with the avg_rating_by_driver, there are few missing values, so I will impute with the median score of 5.0. Since there are about 15% missing values for the avg_rating_of_driver, choice of imputation is a little more delicate. We do not want to throw away almost 15% of the data for missing data in only one column, but imputing could introduce a bias: it is possible that the people who did not leave their driver a rating did not do so because they had a bad experience but felt guilty about leaving a bad rating. In this case imputing with the mean or median would introduce a significant bias. But we might just have to make some assumptions in order to impute the missing data in this column. I will have a look at some of the rows with null in this column and see if there is an obvious pattern. If not, maybe it will be a good sign to go ahead and impute with the average.

In [80]:
#let's view a selection of rows that are missing data in the avg_rating_of_driver col
df.query('`avg_rating_of_driver`.isnull()', engine='python').head(20)

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver,retained
6,Astapor,1,2014-01-24,NaN,1.00,2014-01-25,Android,0.0,False,100.0,3.95,4.0,1
9,Winterfell,1,2014-01-03,NaN,1.00,2014-01-05,Android,0.0,False,0.0,2.37,5.0,1
12,Winterfell,1,2014-01-15,NaN,1.00,2014-03-12,iPhone,0.0,False,100.0,20.29,5.0,1
14,Astapor,1,2014-01-27,NaN,1.00,2014-01-28,Android,0.0,False,100.0,26.01,5.0,1
15,Winterfell,1,2014-01-15,NaN,1.00,2014-01-16,Android,0.0,True,100.0,13.20,5.0,1
20,Winterfell,1,2014-01-31,NaN,1.00,2014-03-16,Android,0.0,False,50.0,2.53,3.0,1
21,Astapor,1,2014-01-09,NaN,1.00,2014-04-29,iPhone,0.0,False,100.0,3.31,5.0,1
22,Winterfell,0,2014-01-24,NaN,1.00,2014-05-02,iPhone,0.0,False,100.0,11.47,5.0,0
25,Winterfell,1,2014-01-02,NaN,1.00,2014-01-03,iPhone,0.0,True,100.0,14.48,5.0,1
28,Winterfell,0,2014-01-19,NaN,1.50,2014-06-08,iPhone,100.0,False,0.0,5.97,5.0,0


In [81]:
df.query('`avg_rating_of_driver`.isnull()', engine='python').tail(20)

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver,retained
49866,Astapor,1,2014-01-14,NaN,1.08,2014-04-10,iPhone,33.3,False,100.0,3.69,4.7,1
49884,Winterfell,1,2014-01-08,NaN,1.00,2014-01-27,iPhone,0.0,True,100.0,18.02,5.0,1
49886,King's Landing,0,2014-01-23,NaN,1.00,2014-06-22,iPhone,0.0,False,0.0,1.91,5.0,0
49891,Astapor,0,2014-01-06,NaN,1.50,2014-03-30,iPhone,100.0,False,0.0,1.07,3.0,0
49912,Winterfell,0,2014-01-14,NaN,1.00,2014-03-17,Android,0.0,False,0.0,1.27,5.0,0
49921,Winterfell,0,2014-01-08,NaN,1.00,2014-06-25,iPhone,0.0,True,100.0,19.74,5.0,0
49926,Astapor,1,2014-01-30,NaN,1.00,2014-01-31,iPhone,0.0,False,100.0,3.82,5.0,1
49927,Winterfell,2,2014-01-24,NaN,1.00,2014-02-16,Android,0.0,False,0.0,3.73,3.0,1
49947,Winterfell,1,2014-01-12,NaN,1.00,2014-01-28,iPhone,0.0,True,100.0,13.74,5.0,1
49951,Winterfell,0,2014-01-29,NaN,1.00,2014-04-16,Android,0.0,True,100.0,4.24,5.0,0


After inspecting a selection of the rows that are missing avg_rating_of_driver data, there is a wide range of variety in the other columns without any obvious pattern (including in the average_rating_by_driver column, so let's make the assumption that it is relatively safe to impute with the mean in this case.

In [85]:
#impute missing phone column with mode - must cast output of mode() as string
df['phone'] = df['phone'].fillna(str(df['phone'].mode()))

In [86]:
#impute missing avg_rating_by_driver with median
df['avg_rating_by_driver'] = df['avg_rating_by_driver'].fillna(df['avg_rating_by_driver'].median())

In [87]:
#impute missing avg_rating_of_driver with mean
df['avg_rating_of_driver'] = df['avg_rating_of_driver'].fillna(df['avg_rating_of_driver'].median())

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   city                    50000 non-null  object        
 1   trips_in_first_30_days  50000 non-null  int64         
 2   signup_date             50000 non-null  datetime64[ns]
 3   avg_rating_of_driver    50000 non-null  float64       
 4   avg_surge               50000 non-null  float64       
 5   last_trip_date          50000 non-null  datetime64[ns]
 6   phone                   50000 non-null  object        
 7   surge_pct               50000 non-null  float64       
 8   ultimate_black_user     50000 non-null  bool          
 9   weekday_pct             50000 non-null  float64       
 10  avg_dist                50000 non-null  float64       
 11  avg_rating_by_driver    50000 non-null  float64       
 12  retained                50000 non-null  int32 

In [89]:
#let's see what unique values there are for the categorical cols: city and phone
display(df['city'].unique(), df['phone'].unique())

array(["King's Landing", 'Astapor', 'Winterfell'], dtype=object)

array(['iPhone', 'Android', '0    iPhone\nName: phone, dtype: object'],
      dtype=object)

There is bit more cleaning to do with the phone column; I will also change any record with '0' to the mode of 'iPhone'. But since there are so few cities and phone types, I can one-hot encode them to keep them in the dataset for the predictive model.

In [108]:
df['phone'] = df['phone'].replace('0', 'iPhone') #df['phone'] = df['phone'].replace('0', str(df['phone'].mode()))

In [106]:
df['phone'].value_counts()

iPhone                                          34582
Android                                         15022
iPhone    iPhone\nName: phone, dtype: object      396
Name: phone, dtype: int64

So now that we have a clean dataset with no missing values, I will create some visualizations to explore the distributions of the data.